In [1]:
import numpy as np
import torch.nn as nn
import pandas as pd
from scalers2 import MinMaxScaler as MMS
from utils import PrCol
from bnn import BNN, BNNDataset
import optuna
import torch
import copy

N_TRIALS = 500
SEED = 42
BATCH_SIZE = 64
DEVICE = torch.device("cuda:0")

Color = PrCol()

dataset_location = "Dataset/bscw_dataset.csv"
SEP = ";"
input_labels = ["mach","aoa"]
output_labels = ["cl","cm"]
fidelity_column_name = "fidelity"
fidelities = ["low","mid","cfd"]
model_name_prefix = "BSCW"

dataset = pd.read_csv(dataset_location,sep=SEP)
scaler = MMS(dataset.drop(columns=[fidelity_column_name], inplace=False),interval=(1,2))
normalized_datasets = []
for fid in fidelities:
    dataset_fidelity = dataset[dataset[fidelity_column_name] == fid]
    dataset_fidelity_norm = scaler.scaleDataframe(dataset_fidelity.drop(columns=[fidelity_column_name], inplace=False))
    normalized_datasets.append(dataset_fidelity_norm)
scaler.save(f"NormalizationData/{model_name_prefix}_norm.pkl")

dataset_lf = BNNDataset(normalized_datasets[0],input_labels,output_labels, device=DEVICE)
dataset_mf = BNNDataset(normalized_datasets[1],input_labels,output_labels, device=DEVICE)
dataset_hf = BNNDataset(normalized_datasets[2],input_labels,output_labels, device=DEVICE)
dataset_all = BNNDataset(pd.concat([normalized_datasets[0], normalized_datasets[1], normalized_datasets[2]], axis=0),input_labels,output_labels,device=DEVICE)
train_lf, valid_lf = dataset_lf.train_val_split(seed=SEED)
train_mf, valid_mf = dataset_mf.train_val_split(seed=SEED)
train_hf, valid_hf, test_hf = dataset_hf.train_val_test_split(train_size=0.09,val_size=0.2,seed=SEED)
max_rows = dataset_hf.data.nlargest(2, ['mach', 'aoa'])
train_hf = BNNDataset(pd.concat([train_hf.data,max_rows]),input_labels,output_labels, device=DEVICE)

In [2]:
bnn_lf_model = BNN(
    in_dim=len(input_labels),
    out_dim=len(output_labels),
    mu = 0,
    std = 0.0126,
    units = [176,176,144,176,176],
    denseOut = False,
    dropout = False,
    device=DEVICE,
    activation=nn.LeakyReLU(),
    model_name="BNN LF"
)
bnn_lf_model.load("AIModels/BSCW_bnn_tl_2.pt")

def objective(trial):
    #mu = trial.suggest_float('mu', -1.5, 1.5, step=0.05)
    #std = trial.suggest_float('std', 0.0001, 0.0996, step=0.0005)
    units = [trial.suggest_int(f'units_{i}', 16, 176, step=16) for i in range(trial.suggest_int('num_layers', 2, 5, step=1))]
    number_of_layers = len(units)
    freezed_layer = trial.suggest_int(f'freezed_layer_from_right', 1, number_of_layers-1, step=1)

    bnn_df_model = copy.deepcopy(bnn_lf_model)
    all_layers = bnn_df_model.getAllLayersName()
    bnn_df_model.setModelGradients(False)
    bnn_df_model.setModelGradients(True, layers=all_layers[-freezed_layer:])

    bnn_df_model.model_name = "BNN TL"
    bnn_df_model.train(
        train_mf,
        valid_mf,
        n_epochs=10000,
        patience=500,
        batch_size = BATCH_SIZE,
        verbose=False,
        plotHistory=False,
        show_loadbar=True
    )
    bnn_df_model_fine_tuned = copy.deepcopy(bnn_df_model) 
    bnn_df_model_fine_tuned.model_name = "BNN TL FT"
    bnn_df_model.setModelGradients(False)
    bnn_df_model.setModelGradients(True,layers=["out_layer"])
    bnn_df_model_fine_tuned.train(
        train_hf,
        valid_hf,
        n_epochs=8000,
        lr=0.001,
        restoreBestModel=True,
        patience=500,
        batch_size = 1,
        earlyStopping=True
    )

    errors, _ = bnn_df_model.testModel(test_set=valid_hf,
                            scaler=scaler,
                            output_labels=output_labels,
                            attempt=100)
    total_errors = []
    
    for out_val in range(len(output_labels)):
        error_on_output_i = np.array([t[out_val] for t in errors])
        # cov_on_output_i = np.array([t[out_val] for t in cov])
        # cov_on_output_i = ((cov_on_output_i) / scaler.scalers[output_labels[out_val]][1]) * 100
        total_errors.append(np.mean(error_on_output_i))
    
    return np.sqrt(np.mean(np.array(total_errors)**2))

# study_name = 'autoencoder_GCN_optimization_1'
# storage_url = 'sqlite:///autoencoder_GCN_optimization_1.db'  # Change to your preferred storage URL
 
# # Create a study object with TPE sampler (Bayesian optimization) and optimize hyperparameters
# study = optuna.create_study(study_name=study_name, storage=storage_url, direction='minimize', sampler=optuna.samplers.TPESampler(seed = seed_value))
# study.optimize(objective, n_trials=20)  

study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed = SEED))
study.optimize(objective, n_trials=N_TRIALS)

# Stampa i risultati migliori
print('Best parameters:', study.best_params)
print('Best performance value:', study.best_value)

[I 2024-05-08 08:40:38,132] A new study created in memory with name: no-name-35c9f1fa-fd63-491e-a17e-1ee1c87a6893


[I 2024-05-08 08:45:15,295] Trial 0 finished with value: 0.02414100617170334 and parameters: {'std': 0.0371, 'num_layers': 5, 'units_0': 144, 'units_1': 112, 'units_2': 32, 'units_3': 32, 'units_4': 16, 'freezed_layer_from_right': 4}. Best is trial 0 with value: 0.02414100617170334.


[I 2024-05-08 08:46:45,293] Trial 1 finished with value: 0.04787240922451019 and parameters: {'std': 0.0601, 'num_layers': 4, 'units_0': 16, 'units_1': 176, 'units_2': 160, 'units_3': 48, 'freezed_layer_from_right': 1}. Best is trial 0 with value: 0.02414100617170334.


[I 2024-05-08 08:49:27,826] Trial 2 finished with value: 0.022239889949560165 and parameters: {'std': 0.0181, 'num_layers': 3, 'units_0': 96, 'units_1': 80, 'units_2': 64, 'freezed_layer_from_right': 2}. Best is trial 2 with value: 0.022239889949560165.


[I 2024-05-08 08:54:18,624] Trial 3 finished with value: 0.02351960726082325 and parameters: {'std': 0.0136, 'num_layers': 3, 'units_0': 80, 'units_1': 96, 'units_2': 144, 'freezed_layer_from_right': 1}. Best is trial 2 with value: 0.022239889949560165.


...

[I 2024-05-09 13:23:21,577] Trial 496 finished with value: 0.023014843463897705 and parameters: {'std': 0.0176, 'num_layers': 2, 'units_0': 176, 'units_1': 160, 'freezed_layer_from_right': 1}. Best is trial 381 with value: 0.01678026281297207.


[I 2024-05-09 13:26:16,908] Trial 497 finished with value: 0.020126307383179665 and parameters: {'std': 0.0146, 'num_layers': 2, 'units_0': 176, 'units_1': 112, 'freezed_layer_from_right': 1}. Best is trial 381 with value: 0.01678026281297207.


[I 2024-05-09 13:28:40,497] Trial 498 finished with value: 0.019584085792303085 and parameters: {'std': 0.0191, 'num_layers': 2, 'units_0': 176, 'units_1': 144, 'freezed_layer_from_right': 1}. Best is trial 381 with value: 0.01678026281297207.


[I 2024-05-09 13:32:34,824] Trial 499 finished with value: 0.01926419883966446 and parameters: {'std': 0.0166, 'num_layers': 2, 'units_0': 176, 'units_1': 128, 'freezed_layer_from_right': 1}. Best is trial 381 with value: 0.01678026281297207.


Best parameters: {'std': 0.0176, 'num_layers': 2, 'units_0': 160, 'units_1': 144, 'freezed_layer_from_right': 1}
Best performance value: 0.01678026281297207


In [3]:
study.best_params

{'std': 0.0176,
 'num_layers': 2,
 'units_0': 160,
 'units_1': 144,
 'freezed_layer_from_right': 1}